In [ ]:
# default_exp preprocessing.data

In [ ]:
#all_slow

# Data

> Classes and functions to load dataset, clean and save for further processing & vocab creation.

In [ ]:
#export
from peptide.basics import *
import pandas as pd
import os

## Analyze Data
- Load all 3 datasets
- Merge into single df?
- Look for class imbalance
- Clean
- Split

In [ ]:
DATA_STORE

'/Users/Vinod/.peptide/datasets'

In [ ]:
os.listdir(f'{DATA_STORE}')

['.DS_Store', 'dna_binding', 'amp', 'acp']

In [ ]:
acp_df = pd.read_csv(f'{DATA_STORE}/acp/train_data.csv')
amp_df = pd.read_csv(f'{DATA_STORE}/amp/all_data.csv')
dna_binding_df = pd.read_csv(f'{DATA_STORE}/dna_binding/train.csv')

In [ ]:
for df in [acp_df, amp_df, dna_binding_df]:
    display(df.head(5))

,sequences,label
0,RRWWRRWRRW,0
1,GWKSVFRKAKKVGKTVGGLALDHYLG,0
2,ALWKTMLKKLGTMALHAGKAALGAAADTISQGTQ,1
3,GLFDVIKKVAAVIGGL,1
4,VAKLLAKLAKKVL,1


,PDBs_code,SequenceID,label
0,AP02484,GMASKAGSVLGKITKIALGAL,1
1,AP02630,NIGLFTSTCFSSQCFSSKCFTDTCFSSNCFTGRHQCGYTHGSC,1
2,AP01427,GAIKDALKGAAKTVAVELLKKAQCKLEKTC,1
3,AP02983,FFGRLKAVFRGARQGWKEHRY,1
4,AP01815,DFGCARGMIFVCMRRCARMYPGSTGYCQGFRCMCDTMIPIRRPPFIMG,1


,code,sequence,label,origin
0,Q6A8L0,MSGHSKWATTKHKKAAIDAKRGKLFARLIKNIEVAARLGGGDPSGN...,1,https://github.com/hfuulgb/PDB-Fusion/tree/mai...
1,Q7V7T9,MIGWLQGQKVEAWQQGTRQGVVLACAGVGYEVQIAPRHLSEMEHGQ...,1,https://github.com/hfuulgb/PDB-Fusion/tree/mai...
2,Q9ZUP2,MARILRNVYSLRSSLFSSELLRRSVVGTSFQLRGFAAKAKKKSKSD...,1,https://github.com/hfuulgb/PDB-Fusion/tree/mai...
3,Q2JVG1,MKCPRCGKQEIRVLESRSAEGGQSVRRRRECMSCGYRFTTYERIEF...,1,https://github.com/hfuulgb/PDB-Fusion/tree/mai...
4,Q9K4Q3,MTKADIIEGVYEKVGFSKKESAEIVELVFDTLKETLERGDKIKISG...,1,https://github.com/hfuulgb/PDB-Fusion/tree/mai...


## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_basics.ipynb.
Converted 01_preprocessing_data.ipynb.
Converted 02_preprocessing_embedding.ipynb.
Converted 03_metrics.ipynb.
Converted index.ipynb.
